# 텍스트 다르기( Handling text data )

## 단어 수준의 원-핫 인코딩

In [1]:
import numpy as np

In [2]:
# 초기 데이터: 각 원소가 샘플입니다
# (이 예에서 하나의 샘플이 하나의 문장입니다. 하지만 문서 전체가 될 수도 있습니다)
samples = [ 'The cat sat on the mat.', 'The dog ate my homework.' ]

In [3]:
# 데이터에 있는 모든 토큰의 인덱스를 구축합니다
token_index = {}
for sample in samples:
    # split() 메서드를 사용해 샘플을 토큰으로 나눕니다.
    # 실전에서는 구둣점과 특수 문자도 사용합니다.
    for word in sample.split():
        if word not in token_index:
            # 단어마다 고유한 인덱스를 할당합니다.
            token_index[ word ] = len( token_index ) + 1
            # 인덱스 0은 사용하지 않습니다.

In [4]:
# 샘플을 벡터로 변환합니다.
# 각 샘플에서 max_length 까지 단어만 사용합니다.
max_length = 10

In [5]:
# 결과를 저장할 배열입니다
results = np.zeros( ( len( samples ), max_length, max( token_index.values() ) + 1 ) )
for i, sample in enumerate( samples ):
    for j, word in list( enumerate( sample.split() ) )[ :max_length ]:
        index = token_index.get( word )
        results[ i, j, index ] = 1.

## 문자 수준 원-핫 인코딩

In [6]:
import string

In [7]:
samples = [ 'The cat sat on the mat.', 'The dog ate my homework.' ]
characters = string.printable  # 출력 가능한 모든 아스키(ASCII) 문자
token_index = dict( zip( characters, range( 1, len( characters ) + 1 ) ) )

In [8]:
max_length = 50
results = np.zeros( ( len( samples ), max_length, max( token_index.values() ) + 1 ) )
for i, sample in enumerate( samples ):
    for j, character in enumerate( sample[ :max_length ] ):
        index = token_index.get( character)
        results[ i, j, index ] = 1.

## 케라스를 사용한 단어 수준의 원-핫 인코딩

In [9]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [10]:
samples = ['The cat sat on the mat.', 'The dog ate my homework.']

# 가장 빈도가 높은 1,000개의 단어만 선택하도록 Tokenizer 객체를 만듭니다.
tokenizer = Tokenizer( num_words = 1000 )
# 단어 인덱스를 구축합니다.
tokenizer.fit_on_texts( samples )

# 문자열을 정수 인덱스의 리스트로 변환합니다.
sequences = tokenizer.texts_to_sequences( samples )

# 직접 원-핫 이진 벡터 표현을 얻을 수 있습니다.
# 원-핫 인코딩 외에 다른 벡터화 방법들도 제공합니다!
one_hot_results = tokenizer.texts_to_matrix( samples, mode = 'binary' )

# 계산된 단어 인덱스를 구합니다.
word_index = tokenizer.word_index
print( 'Found %s unique tokens.' % len( word_index ) )

Found 9 unique tokens.


## 해싱 기법을 사용한 단어 수준의 원-핫 인코딩

- 원-핫 인코딩의 변종 중 하나는 원-핫 해싱 기법입니다. 
- 이 방식은 어휘 사전에 있는 고유한 토큰의 수가 너무 커서 모두 다루기 어려울 때 사용합니다. 
- 각 단어에 명시적으로 인덱스를 할당하고 이 인덱스를 딕셔너리에 저장하는 대신에 단어를 해싱하여 고정된 크기의 벡터로 변환합니다. 
- 일반적으로 간단한 해싱 함수를 사용합니다. 
- 이 방식의 주요 장점은 명시적인 단어 인덱스가 필요 없기 때문에 메모리를 절약하고 온라인 방식으로 데이터를 인코딩할 수 있습니다(전체 데이터를 확인하지 않고 토큰을 생성할 수 있습니다). 
- 한 가지 단점은 해시 충돌입니다. 
- 두 개의 단어가 같은 해시를 만들면 이를 바라보는 머신 러닝 모델은 단어 사이의 차이를 인식하지 못합니다. 
- 해싱 공간의 차원이 해싱될 고유 토큰의 전체 개수보다 훨씬 크면 해시 충돌의 가능성은 감소합니다.

In [11]:
samples = [ 'The cat sat on the mat.', 'The dog ate my homework.' ]

# 단어를 크기가 1,000인 벡터로 저장합니다.
# 1,000개(또는 그이상)의 단어가 있다면 해싱 충돌이 늘어나고 인코딩의 정확도가 감소될 것입니다
dimensionality = 1000
max_length = 10

results = np.zeros( ( len( samples ), max_length, dimensionality ) )
for i, sample in enumerate( samples ):
    for j, word in list(enumerate( sample.split() ) )[ :max_length ]:
        # 단어를 해싱하여 0과 1,000 사이의 랜덤한 정수 인덱스로 변환합니다.
        index = abs( hash( word ) ) % dimensionality
        results[ i, j, index ] = 1.

## 단어 임베딩

In [14]:
from tensorflow.keras.datasets import imdb

from tensorflow.keras import preprocessing
from tensorflow.keras.layers import Embedding

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Embedding

In [16]:
# 특성으로 사용할 단어의 수
max_features = 10000
# 사용할 텍스트의 길이(가장 빈번한 max_features 개의 단어만 사용합니다)
maxlen = 20

# 정수 리스트로 데이터를 로드합니다.
( X_train, y_train ), ( X_test, y_test ) = imdb.load_data( num_words = max_features )

# 리스트를 (samples, maxlen) 크기의 2D 정수 텐서로 변환합니다.
X_train = preprocessing.sequence.pad_sequences( X_train, maxlen = maxlen )
X_test = preprocessing.sequence.pad_sequences( X_test, maxlen = maxlen )

17465344/17464789 [==============================] - 4s 0us/step


In [17]:
model = Sequential()
# 나중에 임베딩된 입력을 Flatten 층에서 펼치기 위해 Embedding 층에 input_length를 지정합니다.
model.add( Embedding( 10000, 8, input_length = maxlen ) )
# Embedding 층의 출력 크기는 (samples, maxlen, 8)가 됩니다.

# 3D 임베딩 텐서를 (samples, maxlen * 8) 크기의 2D 텐서로 펼칩니다.
model.add( Flatten() )

# 분류기를 추가합니다.
model.add( Dense( 1, activation='sigmoid' ) ) 
model.compile( optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = [ 'acc' ] )
model.summary()

history = model.fit( X_train, y_train,
                     epochs = 10,
                     batch_size = 32,
                     validation_split = 0.2 )

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 8)             80000     
_________________________________________________________________
flatten (Flatten)            (None, 160)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 161       
Total params: 80,161
Trainable params: 80,161
Non-trainable params: 0
_________________________________________________________________
Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 4s 184us/sample - loss: 0.6761 - acc: 0.6014 - val_loss: 0.6325 - val_acc: 0.6878
Epoch 2/10
20000/20000 [==============================] - 2s 82us/sample - loss: 0.5509 - acc: 0.7477 - val_loss: 0.5302 - val_acc: 0.7268
Epoch 3/10
20000/20000 [============================

## 사전 훈련된 단어 임베딩 사용

## 모든 내용을 적용하기: 원본 텍스트에서 단어 임베딩까지

 http://mng.bz/0tIo 에서 IMDB 원본 데이터셋을 다운로드하고 압축을 해제
 
- 훈련용 리뷰 하나를 문자열 하나로 만들어 훈련 데이터를 문자열의 리스트로 구성해 보죠. 
- 리뷰 레이블(긍정/부정)도 labels 리스트로 만들겠습니다:

In [18]:
import os

In [22]:
imdb_dir = './datasets/aclImdb'
train_dir = os.path.join( imdb_dir, 'train' )

labels = []
texts = []

for label_type in [ 'neg', 'pos' ]:
    dir_name = os.path.join( train_dir, label_type )
    for fname in os.listdir( dir_name ):
        if fname[ -4: ] == '.txt':
            f = open( os.path.join( dir_name, fname ), encoding = 'utf8' )
            texts.append( f.read() )
            f.close()
            if label_type == 'neg':
                labels.append( 0 )
            else:
                labels.append( 1 )